In [ ]:
import jax
from mujoco import mjx
import mujoco

# Load model
mj_model = mujoco.MjModel.from_xml_string(xml)
mjx_model = mjx.put_model(mj_model)
mjx_data = mjx.make_data(mjx_model)

# Run kinematics to populate data
mjx_data = mjx.forward(mjx_model, mjx_data)

# 1. Mass matrix M(q)
M = mjx.full_m(mjx_model, mjx_data)  # shape: (nv, nv)

# 2. Bias force: C(q,v)*v + G(q)
qfrc_bias = mjx_data.qfrc_bias  # shape: (nv,)

# 3. Get just G(q) by setting velocity to zero
mjx_data_zero_vel = mjx_data.replace(qvel=jax.numpy.zeros_like(mjx_data.qvel))
mjx_data_zero_vel = mjx.forward(mjx_model, mjx_data_zero_vel)
G = mjx_data_zero_vel.qfrc_bias  # Just gravity when v=0

# 4. Jacobian at user-defined point on a body
body_id = mujoco.mj_name2id(mj_model, mujoco.mjtObj.mjOBJ_BODY, "my_body")
point = jax.numpy.array([0.1, 0.0, 0.05])  # Point in global frame
Jp, Jr = mjx.jac(mjx_model, mjx_data, point, body_id)  # (nv, 3), (nv, 3)